Import Neccessary Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import random

Data Parameters

In [ ]:
DATA_DIR = r"C:\Users\user\Downloads\Datasets\Datasets".strip()
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10 
VAL_SPLIT = 0.10
TEST_SPLIT = 0.10
SEED = 42  # For reproducibility

Data Loading and Splitting

In [ ]:
# Create temporary directories for splitting

IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

base_dir = 'splitted_dataset'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Clean up previous runs
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

class_names = sorted([folder for folder in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, folder))])

#class_names = sorted(os.listdir(DATA_DIR))
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    class_path = os.path.join(DATA_DIR, class_name)
    images = [os.path.join(class_path, img) for img in os.listdir(class_path)]
    random.seed(SEED)
    random.shuffle(images)

# Calculate split sizes
    total_images = len(images)
    train_split_idx = int(total_images * (1 - VAL_SPLIT - TEST_SPLIT))
    val_split_idx = int(total_images * (1 - TEST_SPLIT) )

    train_images = images[:train_split_idx]
    val_images = images[train_split_idx:val_split_idx]
    test_images = images[val_split_idx:]

    print(f"Class: {class_name}")
    print(f"  Total images: {total_images}")
    print(f"  Train images: {len(train_images)}")
    print(f"  Validation images: {len(val_images)}")
    print(f"  Test images: {len(test_images)}")

In [ ]:
 # Copy images to respective directories
for img_path in train_images:
        shutil.copy(img_path, os.path.join(train_dir, class_name))
for img_path in val_images:
        shutil.copy(img_path, os.path.join(val_dir, class_name))
for img_path in test_images:
        shutil.copy(img_path, os.path.join(test_dir, class_name))

print("\nDataset split complete.")


In [ ]:
# Data generators for loading images from split directories
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=SEED
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=SEED
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False, # To Keep order for evaluation
    seed=SEED
)
